In [43]:
import pandas as pd
import numpy as np
from skbio.stats.composition import clr_inv as softmax
import warnings
warnings.filterwarnings("ignore")

# Donors

In [44]:
ranks_metafeces = pd.read_table('../../data/donors/meta_metab/latent_dim_3_input_prior_1.00_output_prior_1.00_beta1_0.90_beta2_0.95_ranks.txt')
metagenome=pd.read_table("../../data/donors/metagenome.tsv",sep="\t",index_col=0)
df_metafeces = pd.melt(ranks_metafeces, id_vars='featureid', value_vars=ranks_metafeces.columns,ignore_index=True)
df_metafeces.columns=["HEAD","TAIL","WEIGHT"]
k=1000
df_metafeces = df_metafeces.sort_values('WEIGHT', ascending=False).iloc[:k]

In [45]:
def trimMetagenome(meta_list):
    new_meta=list()
    for m in meta_list:
        new_meta.append(m.split(":")[0])
    return new_meta

In [46]:
def createnode(df,color=["feces","metagenome"]):
    color2shape={
            "metagenome":"ellipse",
            "feces":"ellipse"
    }

    node=pd.DataFrame(columns=['NODE','LABEL','COLOR','SIZE','SHAPE','DESC'])
    for i in df["HEAD"]:
        node=node.append({
            'NODE':i,
            'LABEL':i,
            'COLOR':color[0],
            'SIZE':'20',
            'SHAPE':color2shape[color[0]], 
            'DESC': ''
        },ignore_index=True)

    for j in df["TAIL"]:
        split=j.split(":")
        if(len(split)>1 and color[1]=="metagenome"):
            node=node.append({
                'NODE':j.split(":")[0],
                'LABEL':j.split(":")[0],
                'COLOR':color[1],
                'SIZE':'20',
                'SHAPE':color2shape[color[1]],
                #'DESC': j
                'DESC': j.split(":")[1]
            },ignore_index=True)
        else:
            node=node.append({
                'NODE':j,
                'LABEL':j,
                'COLOR':color[1],
                'SIZE':'20',
                'SHAPE':color2shape[color[1]],
                'DESC':''
            },ignore_index=True)
    
    return node

In [47]:
node_metafeces=createnode(df_metafeces,color=["feces","metagenome"])

In [48]:
metagenomefeces_pwy=set()
for index, row in node_metafeces[node_metafeces["COLOR"]=="metagenome"].iterrows():
    metagenomefeces_pwy.add(row["NODE"])

In [49]:
df_metafeces_taxa=pd.DataFrame(columns=["HEAD","TAIL","WEIGHT"])
node_metafeces_taxa=pd.DataFrame(columns=["NODE","LABEL","COLOR","SIZE","SHAPE","DESC"])

In [50]:
#feces
for m in metagenome.index:
    if "UNINTEGRATED" not in m and "unclassified" not in m:
        pwy=m.split(":")[0]
        if pwy in metagenomefeces_pwy:
            splitpipe=m.split("|")
            if len(splitpipe)==2:
                species=splitpipe[1].split(".")[1]
                row_edge={"HEAD":pwy,"TAIL":species,"WEIGHT":1}
                row_node={"NODE":species,"LABEL":species,"COLOR":"taxa","SIZE":20,"SHAPE":"ellipse","DESC":""}
                df_metafeces_taxa=df_metafeces_taxa.append(row_edge,ignore_index=True)
                node_metafeces_taxa=node_metafeces_taxa.append(row_node,ignore_index=True)

In [51]:
df_metafeces["TAIL"]=trimMetagenome(df_metafeces["TAIL"])

In [52]:
edge=pd.concat([df_metafeces]).drop_duplicates()
node=pd.concat([node_metafeces]).drop_duplicates()

In [53]:
edge.to_csv("../../donors/edge_taxa500.tsv",sep="\t",index=False)
node.to_csv("../../donors/node_taxa500.tsv",sep="\t",index=False)

In [45]:
# pathways=dict()
# for i in list(metagenome.index):
#     for j in tax_set:
#         if j in i:
#             key=i.split("|")[0]
#             if(key in pathways.keys()):
#                 if(j not in pathways[key]):
#                     pathways[key]=pathways[key]+","+j
#             else:
#                 pathways[key]=j

In [50]:
# donor_pathway=pd.DataFrame.from_dict(pathways,orient='index')
# donor_pathway.to_csv("/home/smha118/data1/epilepsy/mmvec/donor/pathway.csv")